In [5]:
import pandas as pd
import numpy as np

In [ ]:
# to open/create a new html file in the write mode
f = open('file239441.html', 'w')

In [ ]:
f

In [2]:
import os

In [3]:
import os.path

In [4]:
from os import listdir

In [5]:
import bs4

In [ ]:
#IMPORTANT
#Code adapted from class, changes: instead of dowloading the output and location files to one folder, 
#I saved them to two different folders, so I don't have to fish for them later with regular expression matching of the file names
#And the path to the source folder and output folders is also different from initial code
# Packages
from os import listdir
from os.path import isfile, join
# Try at home, using 'glob'
import pandas as pd
from bs4 import BeautifulSoup

# Startup code
files = [f for f in listdir('/home/iana/Downloads/Input/') if isfile(join('/home/iana/Downloads/Input/',f))]
print(len(files))

# Processing if only one file: (dedent region)
#f = files[1]
# Processing if multiple files:
for f in files:
    print(f)
    with open('/home/iana/Downloads/Input/'+str(f),'rb') as file:
        soup = BeautifulSoup(file.read(),'html.parser')

    table = soup.find('div','tb s2')
    # classes of each table:
    for t in soup.find_all('table'):
        print(len(t))

    t1 = soup.find_all('table')
    if len(t1)>1:
        t2 = t1[1]

        df = pd.DataFrame(columns=['Names','PersonID','Year','Type','Full'])

        for row in t2.tbody.find_all('tr'):
            #print(row)
            columns = row.find_all('td')
            if len(columns)>0:
                #print(columns)
                names = columns[0].text.strip().replace("\r","").replace("\n","").replace("  "," ")
                year = columns[1].text.strip().replace("\r", "").replace("\n", "").replace("  ", " ")
                type = columns[2].text.strip().replace("\r", "").replace("\n", "").replace("  ", " ")
                office = columns[3].text.strip().replace("\r", "").replace("\n", "").replace("  ", " ")
                full = columns[4].text.strip().replace("\r", "").replace("\n", "").replace("  ", " ")
                c=columns[0].find('a',href=True)
                if c.__str__()!='None':
                    persid = c['href'].replace('../persons/CreatePersonFrames.jsp?PersonID=','')
                else:
                    persid = '0'

                df = df.append({'Names':names,'PersonID':persid,'Year':year,'Type':type, 'Office':office,'Full':full}, ignore_index=True)
        f1 = f.replace('.html','').replace('file','')
        df.to_csv('/home/iana/Downloads/Output/data'+str(f1)+'.csv',index=False)

        l1 = soup.find('ul',{'class':'s2'})
        cols = []
        for row in l1.find_all('li'):
            try:
                cols.append(row.label.text.replace('\xa0',' ').replace("\r", "").replace("\n", "").replace("  ", " ").replace(":", ""))
                last= row.label.text.replace('\xa0',' ').replace("\r", "").replace("\n", "").replace("  ", " ")
            except:
                cols.append(last)
        cols.append('parish')

        df = pd.DataFrame(cols)

        data = []
        for row in l1.find_all('li'):
            text = row.text.replace('\xa0',' ').replace("\r", "").replace("\n", "").replace("  ", " ")
            for c in range(0,len(cols)):
                text = text.replace(cols[c],"")
            data.append(text)

        l2 = soup.find('div',{'class':'ph'})
        data.append(l2.text.replace('\n','').replace('\r',''))
        df2 = pd.DataFrame(data)
        frame = [df,df2]
        final = pd.concat(frame,axis=1)
        final = final.transpose()
        final.to_csv('/home/iana/Downloads/Locations/Location'+str(f1)+'.csv',index=False)


In [6]:
#MY CODE STARTS HERE
#creating a list of ids -- CCED_IDS for files like data*.csv and Location*.csv
#I do this to be able to iterate through them later
#I import library -- glob, it is used for dealing with paths to files
import glob
#here I create a list of all the files in folder Output, which I mentioned earlier contains only files like data2.csv, data3.csv
#as the ids -- CCED_IDS for output files and location files are the same, we need to do this procedure only one time -- getting CCED_ids
a=list(glob.glob("/home/iana/Downloads/Output/*"))
#as the list that we just created contains not only file name but also the whole path to it in my computer, 
#i need to remove two parts from it path and .csv
#it is done in the following two lines
a_stripped=[s.strip('/home/iana/Downloads/Output/') for s in a]
a_clean=[s.strip('.csv') for s in a_stripped]
#in order to be able to concatenate number with a string data or Location, we need to convert the ids to a string format
list_ids=[str(x) for x in a_clean]
#creating our strings to concatenate it with ids -- CCED_IDS in the next lines
data_string = 'data'
locaton_string = 'Location'
#concatenating strings with ids, so we can use the list of resulted strings in following loop to create dataframes with the names according to file names
data_names = [data_string + x for x in list_ids]
location_names = [locaton_string + x for x in list_ids]
#checking if it worked
print(data_names[88])
print(location_names[0])

data1740
Location234791


In [7]:
#creating dataframes for each data and Location files, as we need to merge data and Location files according to their ids
data_path = '/home/iana/Downloads/Output/'

#this loop creates dataframes for each data file from folder output, with the names aka reading all the CSVs and converting them to dataframes
#each created dataframe is named according to the file it was read from
for name in data_names:  
    globals()[name] = pd.read_csv(data_path+name+'.csv')
#at this step we have already all the data files dataframes in Kernel

In [8]:
#now I nead to read and create dataframes from all the location files, dataframes again have the name according to the file they were read from
location_path = '/home/iana/Downloads/Locations/'

#these location files had header as a first row, so we need to make python read the header from the first row
for name in location_names:  
    globals()[name] = pd.read_csv(location_path+name+'.csv', header=[1])
#at this step we have created the dataframes from: all data and location files

In [9]:
#in previous steps I created list with location file names, and these names now stored in the list as strings
#but if i want to perform actions with the frames, i need to change the type of the variables in the list from string to object
#after this step python can understand that elements of the list not just strings, but dataframes that we created earlier
location_dfs = [eval(x) for x in location_names]

In [10]:
#now for each location dataframe I need to add CCED_ID as a column,
#so after I create a dataset with all appointments, I can differentiate between locations
#this loop assigns a new column with CCED_ID to each location dataframe,
for location_df, location_name in zip(location_dfs, location_names):
    location_df['cced_id'] = location_df['cced_id']=location_name

In [11]:
#now I want to merge each data and location dataframes together according to their CCED_ID
#creating string to form from it names of new merged dataframes
merged_string='merged'
merged_names = [merged_string + x for x in list_ids]

In [12]:
#after this step python can understand that elements of the list not just strings, but dataframes that we created earlier
data_dfs = [eval(x) for x in data_names]
location_dfs = [eval(x) for x in location_names]
#merging data and location dataframes, filling missing location values, as it was only one row, with forward fill method
for d,l,m in zip(data_dfs,location_dfs,merged_names): 
    globals()[m] = pd.concat([d, l], axis=1).fillna(method='ffill')

In [13]:
#creating a list of merged dataframes from merged dataframes' names
merged_dfs = [eval(x) for x in merged_names]

In [14]:
#concatenating all merged dataframes together
df_all=pd.concat(merged_dfs)

In [15]:
print(df_all.columns)

Index(['Names', 'PersonID', 'Year', 'Type', 'Full', 'Office', 'County',
       'Diocese (Jurisdiction)', 'Diocese (Geographic)', 'parish', 'cced_id',
       'Contains', 'Contains:', 'Contains:.1', 'Contains:.2', 'In Location',
       'Contains:.3', 'Contains:.4', 'Contains:.5', 'Contains:.6',
       'Contains:.7', 'Contains:.8', 'Contains:.9', 'Contains:.10',
       'Contains:.11', 'Contains:.12', 'Contains:.13', 'Contains:.14',
       'Contains:.15', 'Contains:.16', 'Contains:.17', 'Contains:.18',
       'Contains:.19', 'Contains:.20', 'Contains:.21'],
      dtype='object')


In [16]:
#removing not needed columns
df_all=df_all.drop(['Contains',
       'Contains:', 'Contains:.1', 'Contains:.2', 'Contains:.3',
       'Contains:.4', 'Contains:.5', 'Contains:.6', 'Contains:.7',
       'Contains:.8', 'Contains:.9', 'Contains:.10', 'Contains:.11',
       'Contains:.12', 'Contains:.13', 'Contains:.14', 'Contains:.15',
       'Contains:.16', 'Contains:.17', 'Contains:.18', 'Contains:.19',
       'Contains:.20', 'Contains:.21'], axis=1)

In [17]:
#saving progress to protect from dead Kernel
df_all.to_csv('mathias_all.csv')

In [18]:
df_all.to_excel('mathias_all.xlsx')

In [19]:
df_all=pd.read_csv('mathias_all.csv')

In [20]:
#count of unique people in dataframe
print(df_all['PersonID'].nunique())

131296


In [21]:
#removing string Location from column cced_id
df_all['cced_id'] = df_all['cced_id'].str.replace('Location', '')

In [22]:
#adding population dataframe to Kernel
pop=pd.read_stata('Population_year.dta') 

In [23]:
#unique years in population dataframe
print(pop.year.unique())

[1563 1603 1641 1671 1676 1811 1851 1663 1753 1673 1670 1724 1664 1725
 1788 1679 1672 1666 1674 1762 1688  540 1785 1665 1602 1689 1740 1770
 1662 1685 1738 1650 1735 1779 1758 1667 1768 1783 1791 1795 1789 1797
 1743 1717 1780 1790 1722 1750 1605 1656 1774 1669 1739 1772 1759 1765
 1781 1773 1698 1710 1726 1767 1763 1706 1747 1764 1800 1695 1720 1782
 1709 1778 4000 1170 1850 2300 1280 4250 4910 1792 1786 1794 1643 1621
 1640 1631 1719 1799 1745 1696 1040 1775]


In [24]:
#changing the type of data in Year column, to be able to filter by string lenght after
df_all["Year"]=df_all["Year"].values.astype('str')

In [25]:
#keeping in dataframe only year values that are 4 characters long
df_appointments=df_all[df_all['Year'].str.len()==4]

In [26]:
#changing type of values in Year column back to integers
df_appointments["Year"]=df_appointments["Year"].astype('int')

<ipython-input-26-8c8afc48fadb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appointments["Year"]=df_appointments["Year"].astype('int')


In [27]:
#checking all the unique Year values
appointments_years=df_appointments['Year'].unique().tolist()

In [28]:
appointments_years.sort()

In [29]:
print(appointments_years)

[1454, 1461, 1483, 1520, 1523, 1527, 1528, 1530, 1531, 1532, 1533, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 169

In [30]:
#1525(25)1850 is the goal for the dataframe, thus filtering appointments dataframe accordingly

In [31]:
df_appointments_years=df_appointments[(df_appointments['Year']>=1523)&(df_appointments['Year']<=1854)]

In [32]:
#changing type of year column's values from population dataframe to integer
pop['year']=pop['year'].astype('int')

In [33]:
a=pop['year'].unique().tolist()

In [34]:
a.sort()

In [35]:
#1525(25)1850 is the goal for the dataframe, thus filtering population dataframe accordingly
pop_years=pop[(pop['year']>=1550)&(pop['year']<=1850)]

In [36]:
#creating appointment years list
a_list=df_appointments_years.Year.tolist()

In [37]:
#creating a list of year groups to meet the dataframe requirements 1525(25)1850
group_list=list(range(1550, 1850, 25))

In [38]:
#for each year from appointments finding the index of group_list that the year is closest to
L1 = a_list
L2 = group_list


def find_closest(x):
    l2_diffs = [abs(x - y) for y in L2]
    return l2_diffs.index(min(l2_diffs))

#returns index of the year in the list and the index of the closest group as a tuple
k = [(i, find_closest(x)) for i, x in enumerate(L1)]

print(k)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [39]:
#returns only second elements of tuples 
#creating mask to match the group to years later
mask = [tup[1] for tup in k]

print(mask)

[4, 4, 4, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 2, 5, 5, 5, 6, 9, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 7, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 0, 0, 0, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 7, 7, 7, 7, 7, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 11, 11, 11, 11, 11, 5, 6, 8, 0, 0, 0, 0, 0, 0, 1, 4, 4, 4,

In [40]:
#creating dataframe from mask list
df_mask = pd.DataFrame(mask, columns =['mask_id'])

In [41]:
#creating a list of tuple for matching mask ids with year groups
matching = [(0, 1550), (1, 1575), (2, 1600), (3, 1625), (4, 1650), (5, 1675), (6, 1700), (7, 1725), (8, 1750), (9, 1775), (10, 1800), (11, 1825)]
df_mask["year_group"] = df_mask["mask_id"].map(dict(matching))

In [42]:
#adding to mask dataframe original years
df_mask['year']=list(df_appointments_years['Year'])

In [43]:
#removing duplicates from mask dataframe
df_mask_new = df_mask.drop_duplicates(keep = 'last').reset_index(drop = True)

In [44]:
#merging mask dataframe and appointments dataframe to assign year groups to appointments
appointments = pd.merge(df_appointments_years, df_mask_new, how="outer", left_on=["Year"], right_on=["year"])

In [45]:
#saving progress to protect from dead Kernel
appointments.to_csv('appointments_with_locations_and_year_groups.csv')

In [46]:
#creating a list of unique values from year column from population dataframe
pop_year_list=list(pop_years.year.unique())

In [47]:
#for each unique year from population dataframe finding the index of group_list that the year is closest to
L1 = pop_year_list
L2 = group_list


def find_closest(x):
    l2_diffs = [abs(x - y) for y in L2]
    return l2_diffs.index(min(l2_diffs))

#returns index of the year in the list and the index of the closest group as a tuple
k_new = [(i, find_closest(x)) for i, x in enumerate(L1)]

print(k_new)

[(0, 1), (1, 2), (2, 4), (3, 5), (4, 5), (5, 10), (6, 5), (7, 8), (8, 5), (9, 5), (10, 7), (11, 5), (12, 7), (13, 10), (14, 5), (15, 5), (16, 5), (17, 5), (18, 8), (19, 6), (20, 9), (21, 5), (22, 2), (23, 6), (24, 8), (25, 9), (26, 4), (27, 5), (28, 8), (29, 4), (30, 7), (31, 9), (32, 8), (33, 5), (34, 9), (35, 9), (36, 10), (37, 10), (38, 10), (39, 10), (40, 8), (41, 7), (42, 9), (43, 10), (44, 7), (45, 8), (46, 2), (47, 4), (48, 9), (49, 5), (50, 8), (51, 9), (52, 8), (53, 9), (54, 9), (55, 9), (56, 6), (57, 6), (58, 7), (59, 9), (60, 9), (61, 6), (62, 8), (63, 9), (64, 10), (65, 6), (66, 7), (67, 9), (68, 6), (69, 9), (70, 11), (71, 10), (72, 9), (73, 10), (74, 4), (75, 3), (76, 4), (77, 3), (78, 7), (79, 10), (80, 8), (81, 6), (82, 9)]


In [48]:
#returns only second elements of tuples 
#creating mask to match the group to years later
mask_new = [tup[1] for tup in k_new]

print(mask_new)

[1, 2, 4, 5, 5, 10, 5, 8, 5, 5, 7, 5, 7, 10, 5, 5, 5, 5, 8, 6, 9, 5, 2, 6, 8, 9, 4, 5, 8, 4, 7, 9, 8, 5, 9, 9, 10, 10, 10, 10, 8, 7, 9, 10, 7, 8, 2, 4, 9, 5, 8, 9, 8, 9, 9, 9, 6, 6, 7, 9, 9, 6, 8, 9, 10, 6, 7, 9, 6, 9, 11, 10, 9, 10, 4, 3, 4, 3, 7, 10, 8, 6, 9]


In [49]:
#creating dataframe from mask list
df_mask_new = pd.DataFrame(mask_new, columns =['mask_id'])

In [50]:
#creating a list of tuple for matching mask ids with year groups
matching = [(0, 1550), (1, 1575), (2, 1600), (3, 1625), (4, 1650), (5, 1675), (6, 1700), (7, 1725), (8, 1750), (9, 1775), (10, 1800), (11, 1825)]
df_mask_new["year_group"] = df_mask_new["mask_id"].map(dict(matching))

In [51]:
#adding to mask dataframe original years
df_mask_new['Year']=pop_year_list

In [52]:
#removing duplicates from mask dataframe
df_mask_new1 = df_mask_new.drop_duplicates(keep = 'last').reset_index(drop = True)

In [53]:
#merging mask dataframe and appointments dataframe to assign year groups to appointments
pop_year_groups_new = pd.merge(pop_years, df_mask_new1, how="outer", left_on=["year"], right_on=["Year"])

In [54]:
#saving progress to protect from dead kernel
pop_year_groups_new.to_csv('pop_year_groups_new.csv')

In [55]:
#reading the appointments dataframe that we assigned locations and year groups previously
appointments=pd.read_csv('appointments_with_locations_and_year_groups.csv')

In [56]:
#cleaning all the white spaces from appointment's types, so we don't have duplicates based on different amount of spaces
appointments['type_clean']=appointments['Type'].str.replace(" ","")

In [57]:
#removing not needed columns from appointments dataframe
appointments=appointments.drop(['Unnamed: 0.1','year', 'mask_id'], axis=1)

In [58]:
#reading population dataframe to which we assigned year groups previously
pop_year_groups_new=pd.read_csv('pop_year_groups_new.csv')

In [59]:
#now values of population in the table are floats, changing their type to integer
pop_year_groups_new['Population'] = pop_year_groups_new['Population'].astype(int)

In [60]:
#replacing missing values marked as 0 to numpy NaN values
pop_year_groups_new['Population']=pop_year_groups_new['Population'].replace(0, np.nan)

In [61]:
#in population dataframe there are missing values for population records, and I want to fill them later
#to fill missing population we don't need cced_id, we need c_id, pop, year, year group
pop_small = pop_year_groups_new.drop_duplicates(
  subset = ['C_ID', 'year_group', 'year'],
  keep = 'last').reset_index(drop = True)

In [62]:
#soring values in grouped population dataframe by 3 columns
pop_s = pop_small.sort_values(by=['C_ID', 'year_group', 'year'])
print (pop_s.head())

      Unnamed: 0  cced_id  C_ID  year    lat_MB    lon_MB   latitude  \
335        14686   233218   1.0  1563  52.06274 -0.529209  52.027252   
1346       34773   233218   1.0  1603  52.06274 -0.529209  52.027252   
2357       54860   233218   1.0  1641  52.06274 -0.529209  52.027252   
3048       60807   233218   1.0  1671  52.06274 -0.529209  52.027252   
3411       75601   233218   1.0  1676  52.06274 -0.529209  52.027252   

      longitude  Population  mask_id  year_group  Year  
335   -0.495142       330.0        1        1575  1563  
1346  -0.495142       330.0        2        1600  1603  
2357  -0.495142         NaN        4        1650  1641  
3048  -0.495142       600.0        5        1675  1671  
3411  -0.495142       550.0        5        1675  1676  


In [63]:
#filling missing population value with the average of nearest observations for each city -- C_ID: the nearest previuos and nearest post observation mean
pop_s.loc[:,'Population_clean'] = pop_s.groupby('C_ID')['Population'].apply(lambda group: group.interpolate(method='linear') if np.count_nonzero(np.isnan(group)) < (len(group) - 1) else group)

pop_s.loc[:,'Population_clean'] = pop_s.groupby('C_ID').apply(lambda group: group.interpolate(method='linear', limit_area='outside', limit_direction='both'))

In [64]:
#removing duplicates
pop_s_unique_year_group = pop_s.drop_duplicates(
  subset = ['C_ID', 'year_group'],
  keep = 'last').reset_index(drop = True)

In [65]:
#merging filled with averages population values back to the population dataframe
pop_not_na=pd.merge(pop_year_groups_new, pop_s_unique_year_group[["C_ID","year_group","Population_clean"]], how="left", left_on=["C_ID","year_group"], right_on=["C_ID","year_group"])

In [66]:
#saving progress to protect from dead Kernel
pop_not_na.to_csv('pop_not_na.csv')

In [67]:
#I decided to take into account only appointments that had at least two people present
#Thus, as a first step, I need group by the appointments dataframe by unique location id -- cced_id, year_group and type of the appointment
#and count amount of people present at each appointment, for each location for each type of the appointment
appointments_g=appointments.groupby(['cced_id', 'year_group','type_clean'])['PersonID'].size().reset_index()

In [68]:
#now from counted people at appointments we can filter appointments by having at least two people
appointments_multiple_people=appointments_g[appointments_g['PersonID']>=2]

In [69]:
#now for each unique location and year I count how many appointments with multiple people they had
appointments_by_cced_id_and_year_group=appointments_multiple_people.groupby(['cced_id', 'year_group'])['PersonID'].count().reset_index(name='mult_appointments')

In [70]:
#creating a mapping dataframe for mapping unique location ids -- cced_id to the cities they belong to
mapping = pop_not_na.drop_duplicates(
  subset = ['cced_id', 'C_ID'],
  keep = 'last').reset_index(drop = True)

In [71]:
#merging mapping for cced_ids to C_IDs with appointments dataframe
appo_with_town=pd.merge(appointments_by_cced_id_and_year_group, mapping[["C_ID","cced_id"]], how="left", left_on=["cced_id"], right_on=["cced_id"])

In [72]:
#grouping appointments by city_id -- C_ID, and year group and summing multiple people appointments
appo_year_group_and_city = appo_with_town.groupby(['C_ID', 'year_group'])['mult_appointments'].sum().reset_index(name='appointments')

In [73]:
#saving progress to protect from dead Kernel
appo_year_group_and_city.to_csv('appo_year_group_and_city.csv')

In [74]:
#removing duplicates from population dataframe, as we need for final dataset data to be grouped by year group and city id
pop_city = pop_not_na.drop_duplicates(
  subset = ['C_ID','latitude','longitude','Population_clean','year_group'],
  keep = 'last').reset_index(drop = True)

In [75]:
#merging the population dataframe with appintments dataframe with left join, as previously I filled missing values 
#for population dataframe with averages of neighbor years for each location that had a missing value in population column
appo_with_pop=pd.merge(pop_city, appo_year_group_and_city, how="left", left_on=["C_ID", "year_group"], right_on=["C_ID", "year_group"])

In [76]:
#saving progress to protect from dead Kernel
appo_with_pop.to_csv('appo_with_pop.csv')

In [77]:
appo_with_pop=pd.read_csv('appo_with_pop.csv')

In [78]:
#sorting the final merged dataframe by city id and year group
appo_with_pop_sort = appo_with_pop.sort_values(by=['C_ID', 'year_group'])
print (appo_with_pop_sort.head())

      Unnamed: 0.1  Unnamed: 0  cced_id  C_ID  year    lat_MB    lon_MB  \
335            335       14686   233218   1.0  1563  52.06274 -0.529209   
1345          1345       34773   233218   1.0  1603  52.06274 -0.529209   
2301          2301       54860   233218   1.0  1641  52.06274 -0.529209   
3068          3068       75601   233218   1.0  1676  52.06274 -0.529209   
3617          3617       95688   233218   1.0  1811  52.06274 -0.529209   

       latitude  longitude  Population  mask_id  year_group  Year  \
335   52.027252  -0.495142       330.0        1        1575  1563   
1345  52.027252  -0.495142       330.0        2        1600  1603   
2301  52.027252  -0.495142         NaN        4        1650  1641   
3068  52.027252  -0.495142       550.0        5        1675  1676   
3617  52.027252  -0.495142      1299.0       10        1800  1811   

      Population_clean  appointments  
335              330.0          12.0  
1345             330.0          23.0  
2301             

In [79]:
#removing missing values from final dataframe
appo_with_pop_sort_not_na=appo_with_pop_sort[appo_with_pop_sort['appointments'].notna()]

In [80]:
#saving dataframe for visation of relationship between population and appointments
for_rel_graph=appo_with_pop_sort_not_na[['C_ID','year_group','Population_clean','appointments']]

In [81]:
for_rel_graph.to_csv('for_rel_graph.csv')

# Correlation Matrix

In [82]:
#correlation between population and appointments
cormat = appo_with_pop_sort_not_na[['Population_clean','appointments']].corr()

In [83]:
#the correlation is positive, but very small
#let's look at annual correlation between population and appointments
print(cormat)

                  Population_clean  appointments
Population_clean          1.000000      0.008564
appointments              0.008564      1.000000


In [84]:
#annual correlation between population and appointments
annual_cormat = appo_with_pop_sort_not_na.groupby("year_group")[['Population_clean','appointments']].corr()

In [85]:
#with time correlation changes from negative to positive
print(annual_cormat)

                             Population_clean  appointments
year_group                                                 
1575       Population_clean          1.000000     -0.059994
           appointments             -0.059994      1.000000
1600       Population_clean          1.000000     -0.101926
           appointments             -0.101926      1.000000
1625       Population_clean          1.000000     -1.000000
           appointments             -1.000000      1.000000
1650       Population_clean          1.000000     -0.068377
           appointments             -0.068377      1.000000
1675       Population_clean          1.000000     -0.051813
           appointments             -0.051813      1.000000
1700       Population_clean          1.000000     -0.060644
           appointments             -0.060644      1.000000
1725       Population_clean          1.000000      0.030893
           appointments              0.030893      1.000000
1750       Population_clean          1.0

# Regression absolute values and point estimates

In [86]:
#creating a list with all city ids
C_ID_list=list(appo_with_pop_sort_not_na['C_ID'])

In [87]:
#creating a list with all year groups
year_group_list=list(appo_with_pop_sort_not_na['year_group'])

In [88]:
#creating a list of city ids list and year group ids
arrays = [C_ID_list, year_group_list]
#constructing a new index -- MultiIndex consisting of city id and year group id in a tuple for each row in the dataframe
new_index=pd.MultiIndex.from_arrays(arrays, names=('entity', 'time'))

In [89]:
#assigning new -- MultiIndex to dataframe
appo_multi=appo_with_pop_sort_not_na.set_index(new_index)

In [90]:
print(appo_multi.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'cced_id', 'C_ID', 'year', 'lat_MB',
       'lon_MB', 'latitude', 'longitude', 'Population', 'mask_id',
       'year_group', 'Year', 'Population_clean', 'appointments'],
      dtype='object')


In [91]:
#creating a dataframe with variables of interest
df=appo_multi[['Population_clean','appointments']]

In [92]:
#I constructed the following panel
print(df)

             Population_clean  appointments
entity time                                
1.0    1575             330.0          12.0
       1600             330.0          23.0
       1650             465.0          11.0
       1675             550.0          11.0
       1800            1299.0           9.0
...                       ...           ...
1106.0 1575            1640.0           8.0
       1600            1640.0           1.0
       1650            1640.0           3.0
       1675            1640.0           1.0
       1800            2857.0           4.0

[4498 rows x 2 columns]


In [93]:
#importing a library for building linear model for the panel
from linearmodels.panel import PanelOLS
#specifying the linear model, including the C_ID fixed effects
mod = PanelOLS(df.Population_clean, df.appointments, entity_effects=True)
#clustering the C_ID fixed effects
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/home/iana/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:       Population_clean   R-squared:                        0.0038
Estimator:                   PanelOLS   R-squared (Between):              0.0300
No. Observations:                4394   R-squared (Within):               0.0038
Date:                Fri, Aug 26 2022   R-squared (Overall):              0.0232
Time:                        05:16:27   Log-likelihood                -4.176e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      12.963
Entities:                         970   P-value                           0.0003
Avg Obs:                       4.5299   Distribution:                  F(1,3423)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             3.2508
                            

In [94]:
#importing a library for building linear model for the panel
from linearmodels.panel import PanelOLS
#specifying the linear model, including the C_ID fixed effects
mod = PanelOLS(df.Population_clean, df.appointments, entity_effects=True)
#not-clustering the C_ID fixed effects
res = mod.fit()
print(res)
# for the first model --with clustering city FE, the coefficient infront of appointments is statistically significant at less then 10 percent level
# and for the second model without clustering city fixed effects, appointments coefficient has p-value=0.0003 
#which means that coefficient infront of appointments is
#statistically significant on less then 1 percent level

                          PanelOLS Estimation Summary                           
Dep. Variable:       Population_clean   R-squared:                        0.0038
Estimator:                   PanelOLS   R-squared (Between):              0.0300
No. Observations:                4394   R-squared (Within):               0.0038
Date:                Fri, Aug 26 2022   R-squared (Overall):              0.0232
Time:                        05:18:00   Log-likelihood                -4.176e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      12.963
Entities:                         970   P-value                           0.0003
Avg Obs:                       4.5299   Distribution:                  F(1,3423)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             12.963
                            

/home/iana/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


# regression in logarithms

In [96]:
#adding natural logarithm of population to dataframe
appo_with_pop_sort_not_na['ln_pop']=np.log2(appo_with_pop_sort_not_na['Population_clean'])

<ipython-input-96-42a2db071575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appo_with_pop_sort_not_na['ln_pop']=np.log2(appo_with_pop_sort_not_na['Population_clean'])


In [97]:
#adding natural logarithm of appointments to dataframe
appo_with_pop_sort_not_na['ln_app']=np.log2(appo_with_pop_sort_not_na['appointments'])

<ipython-input-97-62f410ac887a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appo_with_pop_sort_not_na['ln_app']=np.log2(appo_with_pop_sort_not_na['appointments'])


In [98]:
#assigning MultiIndex of tuples C_ID and year group to dataframe
log_multi=appo_with_pop_sort_not_na.set_index(new_index)

In [103]:
print(log_multi)

             Unnamed: 0.1  Unnamed: 0  cced_id    C_ID  year     lat_MB  \
entity time                                                               
1.0    1575           335       14686   233218     1.0  1563  52.062740   
       1600          1345       34773   233218     1.0  1603  52.062740   
       1650          2301       54860   233218     1.0  1641  52.062740   
       1675          3068       75601   233218     1.0  1676  52.062740   
       1800          3617       95688   233218     1.0  1811  52.062740   
...                   ...         ...      ...     ...   ...        ...   
1106.0 1575           407       16359   235417  1106.0  1563  53.953705   
       1600          1417       36446   235417  1106.0  1603  53.953705   
       1650          2351       56533   235417  1106.0  1641  53.953705   
       1675          3094       77274   235417  1106.0  1676  53.953705   
       1800          3688       97361   235417  1106.0  1811  53.953705   

               lon_MB   

In [100]:
#creating a dataframe with variables of interest
df_log=log_multi[['ln_pop','ln_app']]

In [101]:
from linearmodels.panel import PanelOLS
#specifying the linear model in logarithms, including the C_ID fixed effects
mod = PanelOLS(df_log.ln_pop, df_log.ln_app, entity_effects=True)
#clustering C_ID fixed effects
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_pop   R-squared:                        0.0145
Estimator:                   PanelOLS   R-squared (Between):              0.0483
No. Observations:                4394   R-squared (Within):               0.0145
Date:                Fri, Aug 26 2022   R-squared (Overall):              0.0517
Time:                        05:18:18   Log-likelihood                   -5086.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      50.338
Entities:                         970   P-value                           0.0000
Avg Obs:                       4.5299   Distribution:                  F(1,3423)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             41.517
                            

/home/iana/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [102]:
from linearmodels.panel import PanelOLS
#specifying the linear model in logarithms, including the C_ID fixed effects
mod = PanelOLS(df_log.ln_pop, df_log.ln_app, entity_effects=True)
#not-clustering C_ID fixed effects
res = mod.fit()
print(res)

# for both of the runs of model in logarithms --with clustering FE, and without clustering city fixed effects 
#appointments coefficient has p-value=0 which means that coefficient infront of ln_appointments is
#statistically significant on less then 1 percent level

                          PanelOLS Estimation Summary                           
Dep. Variable:                 ln_pop   R-squared:                        0.0145
Estimator:                   PanelOLS   R-squared (Between):              0.0483
No. Observations:                4394   R-squared (Within):               0.0145
Date:                Fri, Aug 26 2022   R-squared (Overall):              0.0517
Time:                        05:18:34   Log-likelihood                   -5086.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      50.338
Entities:                         970   P-value                           0.0000
Avg Obs:                       4.5299   Distribution:                  F(1,3423)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             50.338
                            

/home/iana/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


# Predicting with linear models from scikit-learn

In [69]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [70]:
#dropping rows with NaN values in columns of interest
appo_with_pop_sort_not_na1=appo_with_pop_sort_not_na.dropna(subset=['year_group','appointments','Population_clean'])

In [71]:
#getting dummi-variables for each year group
df_train=pd.get_dummies(appo_with_pop_sort_not_na1, columns=['year_group'])

In [72]:
df_train

,Unnamed: 0.1,Unnamed: 0,cced_id,C_ID,year,lat_MB,lon_MB,latitude,longitude,Population,...,year_group_1600,year_group_1625,year_group_1650,year_group_1675,year_group_1700,year_group_1725,year_group_1750,year_group_1775,year_group_1800,year_group_1825
335,335,14686,233218,1.0,1563,52.062740,-0.529209,52.027252,-0.495142,330.0,...,0,0,0,0,0,0,0,0,0,0
1345,1345,34773,233218,1.0,1603,52.062740,-0.529209,52.027252,-0.495142,330.0,...,1,0,0,0,0,0,0,0,0,0
2301,2301,54860,233218,1.0,1641,52.062740,-0.529209,52.027252,-0.495142,NaN,...,0,0,1,0,0,0,0,0,0,0
3068,3068,75601,233218,1.0,1676,52.062740,-0.529209,52.027252,-0.495142,550.0,...,0,0,0,1,0,0,0,0,0,0
3617,3617,95688,233218,1.0,1811,52.062740,-0.529209,52.027252,-0.495142,1299.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,407,16359,235417,1106.0,1563,53.953705,-1.447715,53.933050,-1.399599,NaN,...,0,0,0,0,0,0,0,0,0,0
1417,1417,36446,235417,1106.0,1603,53.953705,-1.447715,53.933050,-1.399599,NaN,...,1,0,0,0,0,0,0,0,0,0
2351,2351,56533,235417,1106.0,1641,53.953705,-1.447715,53.933050,-1.399599,NaN,...,0,0,1,0,0,0,0,0,0,0
3094,3094,77274,235417,1106.0,1676,53.953705,-1.447715,53.933050,-1.399599,1640.0,...,0,0,0,1,0,0,0,0,0,0


In [73]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 335 to 3688
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      4394 non-null   int64  
 1   Unnamed: 0        4394 non-null   int64  
 2   cced_id           4394 non-null   int64  
 3   C_ID              4394 non-null   float64
 4   year              4394 non-null   int64  
 5   lat_MB            4394 non-null   float64
 6   lon_MB            4394 non-null   float64
 7   latitude          4394 non-null   float64
 8   longitude         4394 non-null   float64
 9   Population        2343 non-null   float64
 10  mask_id           4394 non-null   int64  
 11  Year              4394 non-null   int64  
 12  Population_clean  4394 non-null   float64
 13  appointments      4394 non-null   float64
 14  ln_pop            4394 non-null   float64
 15  ln_app            4394 non-null   float64
 16  year_group_1575   4394 non-null   uint8 

In [74]:
#changing type of variables of interest from string to integer
df_train['Population_clean']=df_train['Population_clean'].astype(int)

In [75]:
df_train['appointments']=df_train['appointments'].astype(int)

In [76]:
#prepairing the dataframe with X -- dependable variables
for_x=df_train[['year_group_1575', 'year_group_1600','year_group_1625','year_group_1650','year_group_1675','year_group_1700','year_group_1725','year_group_1750','year_group_1775','year_group_1800','year_group_1825','appointments']]

In [77]:
#assigning dependable variables and output variable
X=for_x
y=df_train.Population_clean
#splitting X and y for train and test datasets with proportion 75 o 25 percent, applying shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, 
                                                    shuffle=True)

In [78]:
#fitting Random Forest Regression to the Training set
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)

In [79]:
#fiting the model
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [80]:
# Predicting the Test set results for Random Forest Regression
y_pred = regressor.predict(X_test)

In [81]:
# Evaluating the Random Forest Regression algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1658.9531229742336
Mean Squared Error: 24359787.773737222
Root Mean Squared Error: 4935.563572049014


In [82]:
from sklearn.linear_model import LinearRegression

In [83]:
#building a model for linear regression prediction
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [84]:
# Evaluating the linear regression algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1602.8560604882603
Mean Squared Error: 23646422.35072067
Root Mean Squared Error: 4862.758718127054


In [86]:
from sklearn import linear_model
#building a model for Lasso regression prediction
regressor = linear_model.Lasso(alpha=0.1, normalize=True)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [87]:
# Evaluating the Lasso regression algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1601.8285985682396
Mean Squared Error: 23643300.95157728
Root Mean Squared Error: 4862.437758118584


In [88]:
#building a model for LassoLars regression prediction
regressor = linear_model.LassoLars(alpha=0.1, normalize=True)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [89]:
# Evaluating the LassoLars regression algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# I checked prediction MAE, MSE, and RSME for 4 regressor algorithms: Linear Model, RandomForest Regressor, Lasso, and LassoLars
# The best result got LassoLars model
# on the second place Lasso model
# on the third place is Linear model
# and the last is RandomForest Regressor model
#the rating is based on MAE, MSE, and RSME for each model

Mean Absolute Error: 1601.8159054394694
Mean Squared Error: 23643284.816333275
Root Mean Squared Error: 4862.436098946008
